In [2]:
from dotenv import load_dotenv
from pathlib import Path

cwd = Path.cwd().absolute()  # should be project root
load_dotenv(str(cwd / ".env"))

True

In [3]:
from ultralytics import YOLO
from roboflow import Roboflow
import os

pretrained_path = cwd / 'runs' / 'detect' / 'train' / 'weights'

model = YOLO() if not pretrained_path.exists() else YOLO(pretrained_path / 'best.pt')

In [4]:
model.names

{0: 'License_Plate'}

In [5]:
api_key = os.environ["ROBOFLOW_API"]
rf = Roboflow(api_key=api_key)
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(4)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [6]:
import torch

torch.cuda.is_available()

True

In [7]:
dataset_yaml = Path("./License-Plate-Recognition-4/data.yaml").absolute()
results = model.train(
    data=str(dataset_yaml),
    epochs=1,
    imgsz=640,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1050, 4032MiB)
engine/trainer: task=detect, mode=train, model=/home/gbrlbrbs/Projects/cv-cli/runs/detect/train/weights/best.pt, data=/home/gbrlbrbs/Projects/cv-cli/License-Plate-Recognition-4/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sav

train: Scanning /home/gbrlbrbs/Projects/cv-cli/License-Plate-Recognition-4/train/labels.cache... 21173 images, 28 backgrounds, 0 corrupt: 100%|██████████| 21173/21173 [00:00<?, ?it/s]
val: Scanning /home/gbrlbrbs/Projects/cv-cli/License-Plate-Recognition-4/valid/labels.cache... 2046 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2046/2046 [00:00<?, ?it/s]


Plotting labels to /home/gbrlbrbs/Projects/cv-cli/runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/gbrlbrbs/Projects/cv-cli/runs/detect/train6
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      2.47G      1.307      1.086      1.302         11        640: 100%|██████████| 1324/1324 [22:22<00:00,  1.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:43<00:00,  1.47it/s]


                   all       2046       2132      0.944      0.877      0.936      0.574

1 epochs completed in 0.386 hours.
Optimizer stripped from /home/gbrlbrbs/Projects/cv-cli/runs/detect/train6/weights/last.pt, 5.5MB
Optimizer stripped from /home/gbrlbrbs/Projects/cv-cli/runs/detect/train6/weights/best.pt, 5.5MB

Validating /home/gbrlbrbs/Projects/cv-cli/runs/detect/train6/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1050, 4032MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:43<00:00,  1.47it/s]


                   all       2046       2132      0.944      0.877      0.936      0.574
Speed: 0.4ms preprocess, 18.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/gbrlbrbs/Projects/cv-cli/runs/detect/train6


In [9]:
results_val = model.val()

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1050, 4032MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 12,675 gradients, 6.3 GFLOPs


val: Scanning /home/gbrlbrbs/Projects/cv-cli/License-Plate-Recognition-8/valid/labels.cache... 13 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


                   all         13         30      0.899      0.591      0.684      0.256
Speed: 0.6ms preprocess, 26.9ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /home/gbrlbrbs/Projects/cv-cli/runs/detect/val


In [9]:
test_image = cwd / "data" / "test_image_car.jpeg"
result_test = model(str(test_image))
result_test


image 1/1 /home/gbrlbrbs/Projects/cv-cli/data/test_image_car.jpeg: 480x640 1 License_Plate, 44.3ms
Speed: 2.9ms preprocess, 44.3ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'License_Plate'}
 obb: None
 orig_img: array([[[249, 242, 179],
         [250, 239, 181],
         [245, 229, 176],
         ...,
         [ 22,  37,  29],
         [ 25,  40,  32],
         [ 26,  39,  31]],
 
        [[237, 229, 170],
         [254, 243, 186],
         [252, 235, 186],
         ...,
         [ 24,  38,  27],
         [ 23,  37,  26],
         [ 21,  35,  24]],
 
        [[243, 231, 177],
         [249, 234, 185],
         [246, 230, 184],
         ...,
         [ 26,  40,  29],
         [ 23,  37,  26],
         [ 24,  38,  27]],
 
        ...,
 
        [[ 14,  12,  12],
         [ 14,  12,  12],
         [ 14,  12,  12],
         ...,
         [117, 113, 112],
         [119, 115, 114],
         [117, 113, 112]],
 
        [[ 15,  13,  13],
         [ 15,  13,  13],
         [ 15,  13,  13],
         ...,
         [1

In [12]:
result_test[0].save()

'results_test_image_car.jpeg'

In [9]:
model = YOLO(cwd / "runs" / "detect" / "train" / "weights" / "best.pt")